<a href="https://colab.research.google.com/github/javahedi/QuantumStateTomography-ML/blob/main/test_numpy_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
import os
import glob
import csv
import pytest

In [180]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [181]:

path         = "/content/drive/MyDrive/SL_data/"
cvs_names    = glob.glob(f'{path}*.csv')
bank_names   = glob.glob(f'{path}*bank.npy')
weight_names = glob.glob(f'{path}*weights.npy')

angles_list  = []
banks_list   = []
weights_list = []
for id, name in enumerate(cvs_names):
    data    = np.loadtxt( os.path.join(path,cvs_names[id]),delimiter=',',skiprows=1)
    bank    = np.load(bank_names[id],mmap_mode="r")
    weights = np.load(weight_names[id],mmap_mode="r")

    angles_list.append(data)
    banks_list.append(bank[1:])
    weights_list.append(weights[1:])

angles_data   = np.stack(angles_list)
banks_data    = np.stack(banks_list)
weights_data  = np.stack(weights_list)

print(angles_data.shape)
print(banks_data.shape)
print(weights_data.shape)

(1000, 99, 5)
(1000, 99, 100, 2, 2)
(1000, 99, 100)


In [182]:
class InfoGain:
    def information_gain(self, angles, bank_particles, weights):
        angle_dict = {
            "theta": angles[0],
            "phi": angles[1]
        }
        best_guess = np.array(np.einsum('i...,i', bank_particles, weights))
        return self.adaptive_cost_func(angle_dict, bank_particles, weights, best_guess, 1)

    def adaptive_cost_func(self, angles, rhoBank, weights, bestGuess, nQubits):
        meshState = self.angles_to_state_vector(angles, nQubits)
        out = np.einsum('ij,ik->ijk', meshState, meshState.conj())
        K = self.Shannon_entropy(np.einsum('ijk,kj->i', out, bestGuess))
        J = self.Shannon_entropy(np.einsum('ijk,lkj->il', out, rhoBank))
        return np.real(K - np.dot(J, weights))

    def Shannon_entropy(self, prob):
        return np.real(np.sum(-(prob * np.log2(prob)), axis=0))

    def angles_to_state_vector(self, angles, nQubits):
        if nQubits == 1:
            tempMesh = np.array([np.cos(angles["theta"] / 2), np.exp(1j * angles["phi"]) * np.sin(angles["theta"] / 2)])
            meshState = np.array([tempMesh, self.get_opposing_state(tempMesh)])
        else:
            tempMeshA = np.array([np.cos(angles["thetaA"] / 2), np.exp(1j * angles["phiA"]) * np.sin(angles["thetaA"] / 2)])
            tempMeshB = np.array([np.cos(angles["thetaB"] / 2), np.exp(1j * angles["phiB"]) * np.sin(angles["thetaB"] / 2)])
            meshA = np.array([tempMeshA, self.get_opposing_state(tempMeshA)])
            meshB = np.array([tempMeshB, self.get_opposing_state(tempMeshB)])
            meshState = np.array([
                np.kron(meshA[0], meshB[0]),
                np.kron(meshA[0], meshB[1]),
                np.kron(meshA[1], meshB[0]),
                np.kron(meshA[1], meshB[1])
            ])
        return meshState

    def get_opposing_state(self, meshState):
        if meshState[1] == 0:
            return np.array([0, 1], dtype=complex)

        a = 1
        b = -np.conjugate(meshState[0]) / np.conjugate(meshState[1])
        norm = np.sqrt(a * np.conjugate(a) + b * np.conjugate(b))
        oppositeMeshState = np.array([a / norm, b / norm])
        return oppositeMeshState

info_gain = InfoGain()

In [183]:
class InfoGain_Vec:

    def information_gain(self, angles,bank_particles,weights):
        """
        Function is now vectorized in 0th index.
        Takes in angles as [theta,phi] and bank_particles and weigt as how they are given by the file.
        """
        best_guess=np.array(np.einsum('ijkl,ij->ikl',bank_particles,weights))
        return self.adaptive_cost_func(angles,bank_particles,weights,best_guess)


    def adaptive_cost_func(self, angles,rhoBank,weights,bestGuess):

        # Crates projector from angles
        povm=self.angles_to_single_qubit_POVM(angles)
        # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
        K=self.Shannon_entropy(np.einsum('nijk,nkj->ni',povm,bestGuess))
        J=self.Shannon_entropy(np.einsum('nijk,nlkj->nil',povm,rhoBank))
        # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
        return np.real(K-np.einsum("ij,ij->i",J,weights))


    def Shannon_entropy(self, prob):
        """
        Function is now vectorized in 0th index.
        Returns the shannon entorpy of the probability histogram.
        """
        return np.real(np.sum(-(prob*np.log2(prob)),axis=1))

    def angles_to_single_qubit_POVM(self, angles):
        """
        Function is now vectorized in 0th index.
        Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array .
        For single qubit only.
        """
        up_state_vector=np.array([np.cos(angles[:,0]/2),np.exp(1j*angles[:,1])*np.sin(angles[:,0]/2)],dtype=complex)
        up_POVM=np.einsum("in,jn->nij",up_state_vector,up_state_vector.conj())
        return np.einsum('injk->nijk',np.array([up_POVM[:],np.eye(2)-up_POVM[:]],dtype=complex))

info_gain_vec = InfoGain_Vec()

In [184]:
# Select which step in ABME to predict
prediction_index=50

true_angle_np = angles_data[:,prediction_index,3:]
banks_data_np = banks_data[:,prediction_index]
weights_data_np = weights_data[:,prediction_index]
true_info_gain_np=info_gain_vec.information_gain(true_angle_np,banks_data_np,weights_data_np)


In [185]:
print(true_angle_np.shape)
print(banks_data_np.shape)
print(weights_data_np.shape)
print(true_info_gain_np.shape)

(1000, 2)
(1000, 100, 2, 2)
(1000, 100)
(1000,)


In [186]:


class InfoGain_Vec_TF(tf.Module):
    def __init__(self):
        super(InfoGain_Vec_TF, self).__init__()

    def information_gain(self, angles, bank_particles, weights):
        result = tf.numpy_function(self.information_gain_py, [angles, bank_particles, weights], tf.float64)
        return tf.convert_to_tensor(result, dtype=tf.float64)

    def information_gain_py(self, angles,bank_particles,weights):

        best_guess=np.array(np.einsum('ijkl,ij->ikl',bank_particles,weights))
        return self.adaptive_cost_func(angles,bank_particles,weights,best_guess)

    def adaptive_cost_func(self, angles,rhoBank,weights,bestGuess):

        # Crates projector from angles
        povm=self.angles_to_single_qubit_POVM(angles)
        # Computes the entropy of prior and posterior distributions. See 10.1103/PhysRevA.85.052120 for more details.
        K=self.Shannon_entropy(np.einsum('nijk,nkj->ni',povm,bestGuess))
        J=self.Shannon_entropy(np.einsum('nijk,nlkj->nil',povm,rhoBank))
        # Returns the negative values such that it becomes a minimization problem rather than maximization problem.
        return np.real(K-np.einsum("ij,ij->i",J,weights))

    #def Shannon_entropy(self, prob):
    #    return np.real(np.sum(-(prob*np.log2(prob)),axis=1))

    def Shannon_entropy(self, prob):
        epsilon = 1e-10  # Small epsilon value to avoid division by zero
        prob = np.maximum(prob, epsilon)  # Replace zeros with epsilon
        return np.real(np.sum(-(prob * np.log2(prob)), axis=1))

    def angles_to_single_qubit_POVM(self, angles):
        """
        Function is now vectorized in 0th index.
        Takes in measurement angles as dictionaries and returns the spin POVM as 2x2x2 complex array .
        For single qubit only.
        """
        up_state_vector=np.array([np.cos(angles[:,0]/2),np.exp(1j*angles[:,1])*np.sin(angles[:,0]/2)],dtype=complex)
        up_POVM=np.einsum("in,jn->nij",up_state_vector,up_state_vector.conj())
        return np.einsum('injk->nijk',np.array([up_POVM[:],np.eye(2)-up_POVM[:]],dtype=complex))

info_gain_vec_tf = InfoGain_Vec_TF()


In [187]:
print(true_angle_tf.shape)
print(banks_data_tf.shape)
print(weights_data_tf.shape)
print(true_info_gain_tf.shape)

(1000, 2)
(1000, 100, 2, 2)
(1000, 100)
(1000,)


In [188]:
# Set the percentage of data to use for validation
validation_split = 0.2

# Determine the number of samples to use for validation
num_validation_samples = int(angles_data.shape[0] * validation_split)

# Split the data into training and validation sets
train_data_angles  = angles_data[:-num_validation_samples]
train_data_banks   = banks_data[:-num_validation_samples]
train_data_weights = weights_data[:-num_validation_samples]

validation_data_angles  = angles_data[-num_validation_samples:]
validation_data_banks   = banks_data[-num_validation_samples:]
validation_data_weights = weights_data[-num_validation_samples:]

In [189]:
# Set batch size
batch_size = 16

# TensorFlow datasets API
train_dataset      = tf.data.Dataset.from_tensor_slices((train_data_angles, train_data_banks, train_data_weights)).batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_data_angles, validation_data_banks, validation_data_weights)).batch(batch_size)


In [190]:
# Check dimensions of the batches in training dataset
for batch_angles, batch_banks, batch_weights in train_dataset.take(1):
    print("Training Batch Shapes:")
    print("Angles Batch Shape:" , batch_angles.shape)
    print("Banks Batch Shape:"  , batch_banks.shape)
    print("Weights Batch Shape:", batch_weights.shape)

# Check dimensions of the batches in validation dataset
for batch_angles, batch_banks, batch_weights in validation_dataset.take(1):
    print("\nValidation Batch Shapes:")
    print("Angles Batch Shape:" , batch_angles.shape)
    print("Banks Batch Shape:"  , batch_banks.shape)
    print("Weights Batch Shape:", batch_weights.shape)

Training Batch Shapes:
Angles Batch Shape: (16, 99, 5)
Banks Batch Shape: (16, 99, 100, 2, 2)
Weights Batch Shape: (16, 99, 100)

Validation Batch Shapes:
Angles Batch Shape: (16, 99, 5)
Banks Batch Shape: (16, 99, 100, 2, 2)
Weights Batch Shape: (16, 99, 100)


In [191]:

def custom_loss_function(y_true, y_pred, bank_particles, weights, lambda_weight=1.0):

    info_gain_vec_tf = InfoGain_Vec_TF()

    mse_target0 = tf.keras.losses.mean_squared_error(y_true, y_pred)
    mse_target  = tf.reduce_mean(mse_target0, axis=1)

    # average over samples
    true_info_gains = 0
    pred_info_gains = 0
    for i in range(y_true.shape[1]):
        true_info_gains += info_gain_vec_tf.information_gain(y_true[:,i,:], bank_particles[:,i,...], weights[:,i])
        pred_info_gains += info_gain_vec_tf.information_gain(y_pred[:,i,:], bank_particles[:,i,...], weights[:,i])

    true_info_gains = true_info_gains / y_true.shape[1]
    pred_info_gains = pred_info_gains / y_true.shape[1]


    loss_infoGain = 1.0 - (pred_info_gains / true_info_gains)

    total_loss = mse_target + lambda_weight * tf.reduce_mean(loss_infoGain)

    return total_loss


In [192]:
for batch_angles, batch_banks, batch_weights in train_dataset.take(1):

    y_true = batch_angles[...,:2]
    y_pred = batch_angles[...,3:]

    loss = custom_loss_function(y_true, y_pred, batch_banks, batch_weights)


In [193]:
loss

<tf.Tensor: shape=(16,), dtype=float64, numpy=
array([1.82469506, 2.64082368, 1.73837973, 1.94537335, 2.06721568,
       2.13832443, 3.45353623, 2.2985041 , 3.16360485, 3.58051621,
       2.5632466 , 3.27539746, 2.39465389, 1.16598685, 1.35482029,
       2.21495136])>

In [196]:
def custom_loss_function(y_true, y_pred, bank_particles, weights, lambda_weight=1.0):
    info_gain_vec_tf = InfoGain_Vec_TF()

    mse_target0 = tf.keras.losses.mean_squared_error(y_true, y_pred)
    mse_target = tf.reduce_mean(mse_target0, axis=1)

    def compute_info_gain(i):
        true_info_gain = info_gain_vec_tf.information_gain(y_true[:, i, :], bank_particles[:, i, ...], weights[:, i])
        pred_info_gain = info_gain_vec_tf.information_gain(y_pred[:, i, :], bank_particles[:, i, ...], weights[:, i])
        return true_info_gain, pred_info_gain

    true_info_gains, pred_info_gains = tf.map_fn(compute_info_gain, tf.range(y_true.shape[1]), dtype=(tf.float64, tf.float64))

    true_info_gains = tf.reduce_mean(true_info_gains, axis=0)
    pred_info_gains = tf.reduce_mean(pred_info_gains, axis=0)

    loss_infoGain = 1.0 - (pred_info_gains / true_info_gains)

    total_loss = mse_target + lambda_weight * tf.reduce_mean(loss_infoGain)

    return total_loss

In [197]:
for batch_angles, batch_banks, batch_weights in train_dataset.take(1):

    y_true = batch_angles[...,:2]
    y_pred = batch_angles[...,3:]

    loss2 = custom_loss_function(y_true, y_pred, batch_banks, batch_weights)

In [198]:
loss2


<tf.Tensor: shape=(16,), dtype=float64, numpy=
array([1.82469506, 2.64082368, 1.73837973, 1.94537335, 2.06721568,
       2.13832443, 3.45353623, 2.2985041 , 3.16360485, 3.58051621,
       2.5632466 , 3.27539746, 2.39465389, 1.16598685, 1.35482029,
       2.21495136])>